In [23]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'
fp_df_eval_sym_freq_results_train = 'df_eval_sym_freq_results_train'
fp_df_eval_sym_freq_results_validate = 'df_eval_sym_freq_results_validate'
fp_df_eval_sym_freq_results_test = 'df_eval_sym_freq_results_test'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [24]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    # accommodate upto 5 periods of days_lookbacks(i.e. days_lookbacks = [5, 10, 15, 20, 25])
    sym_freq_cnt_15 = []
    sym_freq_cnt_14 = []
    sym_freq_cnt_13 = []
    sym_freq_cnt_12 = []
    sym_freq_cnt_11 = []
    sym_freq_cnt_10 = []
    sym_freq_cnt_9 = []
    sym_freq_cnt_8 = []
    sym_freq_cnt_7 = []
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 15:
            sym_freq_cnt_15.append(_sym)
        elif _freq == 14:
            sym_freq_cnt_14.append(_sym)
        elif _freq == 13:
            sym_freq_cnt_13.append(_sym)
        elif _freq == 12:
            sym_freq_cnt_12.append(_sym)                        
        elif _freq == 11:
            sym_freq_cnt_11.append(_sym)
        elif _freq == 10:
            sym_freq_cnt_10.append(_sym)            
        elif _freq == 9:
            sym_freq_cnt_9.append(_sym)
        elif _freq == 8:
            sym_freq_cnt_8.append(_sym)
        elif _freq == 7:
            sym_freq_cnt_7.append(_sym)  
        elif _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []

    l_sym_freq_cnt.append(sym_freq_cnt_15)
    l_sym_freq_cnt.append(sym_freq_cnt_14)
    l_sym_freq_cnt.append(sym_freq_cnt_13)
    l_sym_freq_cnt.append(sym_freq_cnt_12)    
    l_sym_freq_cnt.append(sym_freq_cnt_11)   
    l_sym_freq_cnt.append(sym_freq_cnt_10)
    l_sym_freq_cnt.append(sym_freq_cnt_9)
    l_sym_freq_cnt.append(sym_freq_cnt_8)
    l_sym_freq_cnt.append(sym_freq_cnt_7)    
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    return l_sym_freq_cnt    

In [25]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt']
col_add1 = ['grp(CAGR)_mean',         'grp(CAGR)_std',         'grp(CAGR)_mean/std']
col_add2 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add3 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add4 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add5 = ['SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI']

col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4 + col_add5
print(f'col_add_total:\n{col_add_total}, total columns: {len(col_add_total)}')

col_add_total:
['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], total columns: 23


#### DANGER: Create a new Dataframe to store run results from below will destroy previous stored data

In [26]:
###################################  To create a new dataframe, select one choice from below
# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_train)
# df

# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_validate)
# df

# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_test)
# df
###################################

#### Select type of run from below: Train, Validate or Test

In [27]:
###################################  Select type of run: train, validate or test
df_eval_sym_freq_results_train = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_train)
# df_eval_sym_freq_results_train
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
run_type = 'train'

# df_eval_sym_freq_results_validate = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
# # df_eval_sym_freq_results_validate
# # Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# # The default split is 0.7, 0.2, 0.1 respectively.
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
# df_train = df_test.copy()
# run_type = 'validate'

# df_eval_sym_freq_results_test = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
# # df_eval_sym_freq_results_test
# # Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# # The default split is 0.7, 0.2, 0.1 respectively.
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
# df_train = df_val.copy()
# run_type = 'test'
###################################

In [28]:
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

len_df_train: 1050, len_df_val: 300, len_df_test: 150 


In [29]:
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
# n_samples = 400  
n_samples = 2

# for training, the number of days to lookback from iloc max-lookback end_train
days_lookbacks = [10, 20, 30]
days_lookbacks.sort()

# number of days from end_train are used to evaluate effectiveness of the training
days_eval = 6


# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

create a sets of iloc lookback slices (start_train:end_train:end_eval), where  
end_train - start_train = days_lookback  
end_eval - end_train = days_eval  
for example,  
if given:  
 n_samples = 2  
 days_lookbacks = [30, 60, 120]  
 days_eval = 10  
a possible result is:  
 max_lookback_slices:  
 [(150, 270, 280), (5, 125, 135)]  
 where 270-150=125-5=max(days_lookbacks), 280-270=135-125=days_eval  
 sets_lookback_slices:  
 [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
  where in a set, 270-240=days_lookbacks[0], 270-210=days_lookbacks[1], 270-150=days_lookbacks[2]  
  and 270, i.e. end_train, is constant for the set  

In [30]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(502, 532, 538), (278, 308, 314)]
sets_lookback_slices:
[[(522, 532, 538), (512, 532, 538), (502, 532, 538)], [(298, 308, 314), (288, 308, 314), (278, 308, 314)]]


Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The performance metrics are calculated based on slices in sets_lookback_slices.  The first two numbers are ilocs for training. The last two numbers are ilocs for evaluation.

In [31]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    _df = df_train.iloc[start_train:end_train]
    date_start_train = _df.index[0].strftime('%Y-%m-%d')
    date_end_train = _df.index[-1].strftime('%Y-%m-%d')

    if verbose:
      print(f'days lookback:    {lookback}')
      print(f'lb_slices:        {lb_slices}')
      print(f'lb_slice:         {lb_slice}')
      print(f'days eval:        {d_eval}')    
      print(f'iloc_start_train: {start_train}')
      print(f'iloc_end_train:   {end_train}')
      print(f'date_start_train: {date_start_train}')
      print(f'date_end_train:   {date_end_train}')


    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

days lookback:    10
lb_slices:        [(522, 532, 538), (512, 532, 538), (502, 532, 538)]
lb_slice:         (522, 532, 538)
days eval:        6
iloc_start_train: 522
iloc_end_train:   532
date_start_train: 2019-04-16
date_end_train:   2019-04-30
perf_ranks: {'period-10': {'r_CAGR/UI': array(['PI', 'MGI', 'SOHU', 'PRO', 'USB', 'YELP', 'RCII', 'QCOM', 'HELE',
       'MA', 'SAP', 'STZ', 'AZTA', 'SBS', 'UFPI', 'MSFT', 'KDP', 'RNG',
       'NOW', 'F'], dtype=object), 'r_CAGR/retnStd': array(['PI', 'MGI', 'SOHU', 'RCII', 'QCOM', 'HELE', 'PRO', 'SBS', 'UFPI',
       'AZTA', 'PLAY', 'YELP', 'SAP', 'VRNS', 'AXSM', 'STZ', 'RNG', 'KDP',
       'NOW', 'FDP'], dtype=object), 'r_retnStd/UI': array(['MA', 'USB', 'YELP', 'PI', 'PRO', 'STZ', 'MGI', 'SAP', 'MSFT',
       'SYF', 'TAP', 'AZTA', 'CSL', 'HELE', 'RTX', 'SOHU', 'EVT', 'QCOM',
       'F', 'PAYX'], dtype=object)}}
most_common_syms: [('PI', 3), ('MGI', 3), ('SOHU', 3), ('PRO', 3), ('YELP', 3), ('QCOM', 3), ('HELE', 3), ('SAP', 3), ('STZ', 3), (

In [32]:
for item, top_set_syms_n_freq in enumerate(grp_top_set_syms_n_freq):
  l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
  if verbose:
    print(f'{item + 1} of {len(grp_top_set_syms_n_freq)} max_lookback_slices: {max_lookback_slices}')
    print(f'max_lookback_slice: {max_lookback_slices[item]}')
    print(f'sym_freq_15: {l_sym_freq_cnt[0]}')
    print(f'sym_freq_14: {l_sym_freq_cnt[1]}')
    print(f'sym_freq_13: {l_sym_freq_cnt[2]}')
    print(f'sym_freq_12: {l_sym_freq_cnt[3]}')
    print(f'sym_freq_11: {l_sym_freq_cnt[4]}')
    print(f'sym_freq_10: {l_sym_freq_cnt[5]}')
    print(f'sym_freq_9: {l_sym_freq_cnt[6]}')
    print(f'sym_freq_8: {l_sym_freq_cnt[7]}')
    print(f'sym_freq_7: {l_sym_freq_cnt[8]}')
    print(f'sym_freq_6: {l_sym_freq_cnt[9]}')
    print(f'sym_freq_5: {l_sym_freq_cnt[10]}')
    print(f'sym_freq_4: {l_sym_freq_cnt[11]}')
    print(f'sym_freq_3: {l_sym_freq_cnt[12]}')
    print(f'sym_freq_2: {l_sym_freq_cnt[13]}\n')  

1 of 2 max_lookback_slices: [(502, 532, 538), (278, 308, 314)]
max_lookback_slice: (502, 532, 538)
sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: ['QCOM']
sym_freq_8: ['PI', 'STZ']
sym_freq_7: ['HELE', 'MGI']
sym_freq_6: ['AZTA', 'F', 'PRO']
sym_freq_5: ['DIS', 'PPC']
sym_freq_4: ['ADNT', 'STNG', 'XPO']
sym_freq_3: ['AXSM', 'CAG', 'CPRX', 'CSL', 'EVT', 'FIVE', 'KMX']
sym_freq_2: []

2 of 2 max_lookback_slices: [(502, 532, 538), (278, 308, 314)]
max_lookback_slice: (278, 308, 314)
sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: []
sym_freq_8: ['RARE']
sym_freq_7: []
sym_freq_6: ['AVXL', 'CYRX', 'MOV', 'SCVL', 'SIG', 'TNDM', 'TTD', 'WWE']
sym_freq_5: ['FL', 'MED', 'SPTN']
sym_freq_4: ['DDS', 'FIVE', 'NSTG', 'WSM']
sym_freq_3: ['CATO', 'DECK', 'KN', 'LULU']
sym_freq_2: []



In [61]:
from myUtils import symb_perf_stats_vectorized_v8

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):
  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  if verbose:
    print(f'sym_freq_15: {l_sym_freq_cnt[0]}')
    print(f'sym_freq_14: {l_sym_freq_cnt[1]}')
    print(f'sym_freq_13: {l_sym_freq_cnt[2]}')
    print(f'sym_freq_12: {l_sym_freq_cnt[3]}')
    print(f'sym_freq_11: {l_sym_freq_cnt[4]}')
    print(f'sym_freq_10: {l_sym_freq_cnt[5]}')
    print(f'sym_freq_9: {l_sym_freq_cnt[6]}')
    print(f'sym_freq_8: {l_sym_freq_cnt[7]}')
    print(f'sym_freq_7: {l_sym_freq_cnt[8]}')
    print(f'sym_freq_6: {l_sym_freq_cnt[9]}')
    print(f'sym_freq_5: {l_sym_freq_cnt[10]}')
    print(f'sym_freq_4: {l_sym_freq_cnt[11]}')
    print(f'sym_freq_3: {l_sym_freq_cnt[12]}')
    print(f'sym_freq_2: {l_sym_freq_cnt[13]}\n')  


  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]

  (
    _symbols,
    _period_yr,
    _retn,
    _DD,
    _UI,
    _MDD,
    _retnMean,
    _retnStd,
    _retnStd_div_UI,
    _CAGR,
    _CAGR_div_retnStd,
    _CAGR_div_UI,
    SPY_retnStd_d_UI,     
    SPY_CAGR,
    SPY_CAGR_d_retnStd,
    SPY_CAGR_d_UI,
  ) = symb_perf_stats_vectorized_v8(df_SPY)  

  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>17,.3f}, {SPY_CAGR_d_retnStd[0]:>17,.3f}, {SPY_CAGR_d_UI[0]:>17.1f}, {SPY_CAGR[0]:>17.1f}')

  # drop last list, with frequency count 2 or less, in l_sym_freq_cnt from zip
  # zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  zip_cnt_n_syms = zip([15,14,13,12,11,10,9,8,7,6,5,4,3], l_sym_freq_cnt[:-1])
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df_train[start_eval:end_eval][syms]      

      if verbose:
        print(f'start_eval: {start_eval},  date: {d_end_train}')
        print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df_train[start_eval:end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')

      (
        _symbols,
        _period_yr,
        _retn,
        _DD,
        _UI,
        _MDD,
        _retnMean,
        _retnStd,
        _retnStd_div_UI,
        _CAGR,
        _CAGR_div_retnStd,
        _CAGR_div_UI,
        grp_retnStd_d_UI,     
        grp_CAGR,
        grp_CAGR_d_retnStd,
        grp_CAGR_d_UI,
      ) = symb_perf_stats_vectorized_v8(df_eval)  
      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>17,.3f}, {grp_retnStd_d_UI[1]  :>17,.3f}, {grp_retnStd_d_UI[2]  :>17,.3f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>17,.3f}, {grp_CAGR_d_retnStd[1]:>17,.3f}, {grp_CAGR_d_retnStd[2]:>17,.3f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>17,.3f}, {grp_CAGR_d_UI[1]     :>17,.3f}, {grp_CAGR_d_UI[2]     :>17,.3f}')
      print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>17,.3f}, {grp_CAGR[1]          :>17,.3f}, {grp_CAGR[2]          :>17,.3f}')

      if store_results:  # record results to df
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, sym_freq_cnt]
        row_add1      = [grp_CAGR[0],           grp_CAGR[1],           grp_CAGR[2]]
        row_add2      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add3      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add4      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add5      = [SPY_CAGR[0], SPY_CAGR_d_UI[0], SPY_CAGR_d_retnStd[0], SPY_retnStd_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4 + row_add5
        print(f'row_add_total: {row_add_total}')

        if run_type == 'train':
          df_eval_sym_freq_results_train.loc[len(df_eval_sym_freq_results_train)] = row_add_total
          print(f'appended row_add to df_eval_sym_freq_results_train:\n{row_add_total}\n')
        elif run_type == 'validate':
          df_eval_sym_freq_results_validate.loc[len(df_eval_sym_freq_results_validate)] = row_add_total       
          print(f'appended row_add to df_eval_sym_freq_results_validate:\n{row_add_total}\n')
        elif run_type == 'test':
          df_eval_sym_freq_results_test.loc[len(df_eval_sym_freq_results_test)] = row_add_total       
          print(f'appended row_add to df_eval_sym_freq_results_test:\n{row_add_total}\n')
        else:
          msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
          raise SystemExit(msg_stop )                             

  print(f'\n{"="*200}\n{"="*200}\n\n') 

z_grp_top_set_syms_n_freq:
1 of 2 max_lookback_slice
max_lookback_slice: (502, 532, 538)
max lookback dates: 2019-03-19, 2019-05-01, 2019-05-09
df_eval dates (inclusive): 2019-05-01 - 2019-05-08
top_set_syms_n_freq: [('QCOM', 9), ('PI', 8), ('STZ', 8), ('HELE', 7), ('MGI', 7), ('AZTA', 6), ('F', 6), ('PRO', 6), ('DIS', 5), ('PPC', 5), ('ADNT', 4), ('STNG', 4), ('XPO', 4), ('AXSM', 3), ('CAG', 3), ('CPRX', 3), ('CSL', 3), ('EVT', 3), ('FIVE', 3), ('KMX', 3)]

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: ['QCOM']
sym_freq_8: ['PI', 'STZ']
sym_freq_7: ['HELE', 'MGI']
sym_freq_6: ['AZTA', 'F', 'PRO']
sym_freq_5: ['DIS', 'PPC']
sym_freq_4: ['ADNT', 'STNG', 'XPO']
sym_freq_3: ['AXSM', 'CAG', 'CPRX', 'CSL', 'EVT', 'FIVE', 'KMX']
sym_freq_2: []


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR:             0.674,           -62.258,             -41.9,              -0.5
start_eval: 532,  date: 2019-05-01
end_eval:   538,  date: 2019-05-

In [62]:
if store_results:  # record results to df
  if run_type == 'train':
    pickle_dump(df_eval_sym_freq_results_train, path_data_dump, fp_df_eval_sym_freq_results_train)
    print(f'Save results to: {fp_df_eval_sym_freq_results_train}')
  elif run_type == 'validate':
    pickle_dump(df_eval_sym_freq_results_validate, path_data_dump, fp_df_eval_sym_freq_results_validate)   
    print(f'Save results to: {fp_df_eval_sym_freq_results_validate}')
  elif run_type == 'test':
    pickle_dump(df_eval_sym_freq_results_test, path_data_dump, fp_df_eval_sym_freq_results_test)   
    print(f'Save results to: {fp_df_eval_sym_freq_results_test}')
  else:
    msg_stop = f"ERROR: Result was not saved. run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
    raise SystemExit(msg_stop )  

Save results to: df_eval_sym_freq_results_train


In [64]:
if verbose:
  from IPython.display import display, HTML
  df = df_eval.copy()
  df_SPY = df_SPY.copy()  
  _my_df = [df_eval, df_SPY]
  for _df in _my_df:
    # print(f'_df:\n{_df}')
    (
      symbols,
      period_yr,
      retn,
      DD,
      UI,
      MDD,
      retnMean,
      retnStd,
      retnStd_div_UI,
      CAGR,
      CAGR_div_retnStd,
      CAGR_div_UI,
      grp_retnStd_div_UI,     
      grp_CAGR,
      grp_CAGR_div_retnStd,
      grp_CAGR_div_UI,
    ) = symb_perf_stats_vectorized_v8(_df)

    # display(HTML(_df.to_html()))
    print(f'_df:\n{_df}\n')
    print(f'symbols:\n{symbols}\n')
    print(f'period_yr:\n{period_yr}\n')
    print(f'retn:\n{retn}\n')
    print(f'DD:\n{DD}\n')
    print(f'UI:\n{UI}\n')
    print(f'MDD:\n{MDD}\n')
    print(f'retnMean:\n{retnMean}\n')
    print(f'retnStd:\n{retnStd}\n')
    print(f'retnStd_div_UI:\n{retnStd_div_UI}\n')
    print(f'CAGR:\n{CAGR}\n')
    print(f'CAGR_div_retnStd:\n{CAGR_div_retnStd}\n')
    print(f'CAGR_div_UI:\n{CAGR_div_UI}\n')
    print(f'grp_retnStd_div_UI:\n{grp_retnStd_div_UI}\n')
    print(f'grp_CAGR_div_retnStd:\n{grp_CAGR_div_retnStd}\n')    
    print(f'grp_CAGR_div_UI:\n{grp_CAGR_div_UI}\n')
    print(f'grp_CAGR:\n{grp_CAGR}\n')                

_df:
                 CATO        DECK     KN        LULU
Date                                                
2018-06-08  19.026583  121.650002  15.48  122.019997
2018-06-11  19.073879  121.250000  15.81  122.370003
2018-06-12  19.239389  120.800003  15.64  124.959999
2018-06-13  19.349737  119.330002  15.82  125.129997
2018-06-14  19.333975  119.099998  15.93  125.879997
2018-06-15  19.862055  118.809998  15.87  128.369995

symbols:
Index(['CATO', 'DECK', 'KN', 'LULU'], dtype='object')

period_yr:
0.01984126984126984

retn:
                CATO      DECK        KN      LULU
Date                                              
2018-06-08  0.000000  0.000000  0.000000  0.000000
2018-06-11  0.002486 -0.003288  0.021318  0.002868
2018-06-12  0.008677 -0.003711 -0.010753  0.021165
2018-06-13  0.005736 -0.012169  0.011509  0.001360
2018-06-14 -0.000815 -0.001927  0.006953  0.005994
2018-06-15  0.027314 -0.002435 -0.003767  0.019781

DD:
                CATO      DECK        KN      LULU
Date